# Filter AFI and CHA Code Zeolite from OSDA_ZEO.xlsx

In [1]:
%cd ..

/home/hudongcheng/Desktop/bo_osda_generator


In [2]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
from tqdm import tqdm
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
import torch.backends.cudnn as cudnn

# import custom modules
from models.clamer import *
from utils.utils import *
from datasets.data_loader import *
from utils.plot_figures import *
from utils.metrics import *
from utils.build_vocab import *

In [3]:
# load config
save_best_weight_path = './checkpoints/'
sample_loss_history = []
now = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
data_file = r'./data/OSDA_ZEO.xlsx'
batch_size = 64

In [4]:
# feature the zeolite
def featurize_zeolite(row):
    features = [row['FD'], row['max_ring_size'], row['channel_dim'], row['inc_vol'], row['accvol'], row['maxarea'],
                row['minarea']]
    if any([pd.isnull(x) for x in features]):
        # print('Problem: ', row['Code'], 'Missing Information in Data')
        return None
    else:
        return np.array(features, dtype=float)

# merge the synthesis conditions
def featurize_synthesis(row):
    seeds = ['seed', 'SAPO-56 seeds', 'SSZ-57', 'FAU', 'seeded with magadiite', 'seeds']
    solvents = ['ethylene glycol', 'hexanol', '2-propanol', 'triethylene glycol', 'triglycol',
                'polyethylene glycol', 'n-hexanol', 'glycol', 'propane-1,3-diol', 'butanol',
                'glycerol', 'isobutylamine', 'tetraethylene glycol', '1-hexanol',
                'sec-butanol', 'iso-butanol', 'ethylene glycol monomethyl ether', 'ethanol']
    acids = ['H2SO4', 'acetic acid', 'oxalic acid', 'succinic acid', 'arsenic acid', 'HNO3', 'HCl',
             'SO4']
    frameworks = ['Co', 'Mn', 'Cu', 'Zn', 'Cd', 'Cr', 'V', 'Ce', 'Nd', 'Sn', 'Zr', 'Ni',
                  'S', 'Sm', 'Dy', 'Y', 'La', 'Gd', 'In', 'Nb', 'Te', 'As', 'Hf', 'W',
                  'Se']
    common_frameworks = ['Si', 'Al', 'P', 'Ge', 'B', 'Ti', 'Ga', 'Fe']
    cations = ['Mg', 'Rb', 'Li', 'Cs', 'Sr', 'Ba', 'Be', 'Ca']
    common_cations = ['Na', 'K']
    bad = ['pictures', 'need access', 'also called azepane', 'SMILES code']
    syns = [x.strip() for x in [row['syn1'], row['syn2'], row['syn3'], row['syn4'], row['syn5'],
                                row['syn6'], row['syn7'], row['syn8']] if not pd.isnull(x)]
    if not syns:
        return None
    syn_vector = []
    for c in common_frameworks:
        if c in syns:
            syn_vector.append(1)
        else:
            syn_vector.append(0)
    for c in common_cations:
        if c in syns:
            syn_vector.append(1)
        else:
            syn_vector.append(0)
    if 'F' in syns:
        syn_vector.append(1)
    else:
        syn_vector.append(0)
    frame, cat, seed, solv, acid, oth = 0, 0, 0, 0, 0, 0
    for s in syns:
        if s in frameworks:
            frame = 1
        elif s in cations:
            cat = 1
        elif s in seeds:
            seed = 1
        elif s in solvents:
            solv = 1
        elif s in acids:
            acid = 1
        elif s.count(' ') < 2 and s not in bad and len(s) > 2:
            oth = 1
    syn_vector.extend([frame, cat, seed, solv, acid, oth])
    return np.array(syn_vector, dtype=float)

# data augmentation
def data_augment(augment, data):
    smiles_aug, zeo_features_aug, syn_features_aug, codes_aug = [], [], [], []
    for i, row in data.iterrows():
        if ' + ' not in row['smiles']:  # only look at single-template synthesis
            zeo = featurize_zeolite(row=row)
            syn = featurize_synthesis(row=row)
            if zeo is not None and syn is not None:
                if augment:
                    new_smiles = []
                    m = Chem.MolFromSmiles(row['smiles'])
                    for i in range(100):  # randomize smiles string up to 100 times
                        try:
                            rand_smile = Chem.MolToSmiles(m, canonical=False, doRandom=True, isomericSmiles=False)
                            rand_mol = Chem.MolFromSmiles(rand_smile)
                            if m is not None and rand_smile not in new_smiles:
                                new_smiles.append(rand_smile)
                        except:
                            # print('Problem:', row['smiles'], 'could not be randomized')
                            break
                    for smile in new_smiles:
                        smiles_aug.append(smile)
                        zeo_features_aug.append(zeo)
                        syn_features_aug.append(syn)
                        codes_aug.append(row['Code'])
                else:
                    smiles_aug.append(row['smiles'])
                    zeo_features_aug.append(zeo)
                    syn_features_aug.append(syn)
                    codes_aug.append(row['Code'])
    return smiles_aug, zeo_features_aug, syn_features_aug, codes_aug

# normalize the data
def norm(data):
    data_max, data_min = np.tile(np.max(data, axis=0), (data.shape[0], 1)), np.tile(
        np.min(data, axis=0), (data.shape[0], 1))
    data_norm = (data - data_min) / (data_max - data_min)
    return data_norm

# filter the data
def data_sift(smiles, zeo_vecs, syn_vecs, codes):
    cant = ['a', 't', ' ', 'i', 'd', 'e', 'f', 'y', 'u']
    new_smile, new_zeo, new_syn, new_codes = [], [], [], []
    for s, z, v, d in zip(smiles, zeo_vecs, syn_vecs, codes):
        if 'Si' in s:
            continue
        found = False
        for c in s:
            if c in cant:
                found = True
        if not found:
            new_zeo.append(z)
            new_syn.append(v)
            new_smile.append(s)
            new_codes.append(d)

    return new_smile, np.array(new_zeo), np.array(new_syn), new_codes

# split the data
def data_split(split, unique_codes, smiles, zeo_vectors, syn_vectors, codes):
    train_smiles, train_zeo, train_syn, train_codes = [], [], [], []
    test_smiles, test_zeo, test_syn, test_codes = [], [], [], []
    if split is None:
        train_smiles = smiles
        train_zeo = zeo_vectors
        train_syn = syn_vectors
        train_codes = codes
    elif split == 'random':
        unique_smiles = list(np.unique(smiles))
        print(len(unique_smiles))
        test_indices = []
        random.shuffle(unique_smiles)
        test_smiles_unique = unique_smiles[:round(0.2 * len(unique_smiles))]  # 20% held out set
        print(len(test_smiles_unique))
        for t in test_smiles_unique:
            for i, s in enumerate(smiles):
                if t == s:
                    test_indices.append(i)
        print(len(test_indices))
        for i, (s, z, v, c) in enumerate(zip(smiles, zeo_vectors, syn_vectors, codes)):
            if i in test_indices:
                test_smiles.append(s)
                test_zeo.append(z)
                test_syn.append(v)
                test_codes.append(c)
            else:
                train_smiles.append(s)
                train_zeo.append(z)
                train_syn.append(v)
                train_codes.append(c)
    else:
        if split not in unique_codes:
            print('Problem:', split, 'not a zeolite in the data')
        else:
            for i, (s, z, v, c) in enumerate(zip(smiles, zeo_vectors, syn_vectors, codes)):
                if split == c:
                    test_smiles.append(s)
                    test_zeo.append(z)
                    test_syn.append(v)
                    test_codes.append(c)
                else:
                    train_smiles.append(s)
                    train_zeo.append(z)
                    train_syn.append(v)
                    train_codes.append(c)
    return train_smiles, train_zeo, train_syn, train_codes, test_smiles, test_zeo, test_syn, test_codes


In [5]:
# read data
data = pd.read_excel(data_file, engine='openpyxl')
smiles_aug, zeo_features_aug, syn_features_aug, codes_aug = data_augment(augment=False, data=data)

zeo_features = np.array(zeo_features_aug)
zeo_features = norm(zeo_features)

smiles, zeo_vectors, syn_vectors, codes = data_sift(smiles_aug, zeo_features, syn_features_aug, codes_aug)
print(len(set(smiles)), len(set(codes)))
unique_codes = ['AFI']
split = "AFI"
_, _, _, _, test_smiles, test_zeo, test_syn, _ = data_split(split, unique_codes,
                                                                        smiles, zeo_vectors, syn_vectors, codes)

print('len of test_smiles:', len(test_smiles))
print('len of test_zeo:', len(test_zeo))
print('len of test_syn:', len(test_syn))

print('type of test_smiles:', type(test_smiles))
print('type of test_zeo:', type(test_zeo))
print('type of test_syn:', type(test_syn))

test_smiles = test_smiles * 250
test_zeo = test_zeo * 250  # * 121 42
test_syn = test_syn * 250  # * 121 42
AFI_smiles = test_smiles[0:1000]
AFI_zeo = test_zeo[0:1000]
AFI_syn = test_syn[0:1000]

# save AFI_smiles, AFI_zeo, AFI_syn to data_AFI folder as 3 csv files
AFI_smiles = pd.DataFrame(AFI_smiles)
AFI_zeo = pd.DataFrame(AFI_zeo)
AFI_syn = pd.DataFrame(AFI_syn)
AFI_smiles.to_csv('./data_AFI/AFI_smiles.csv', index=False)
AFI_zeo.to_csv('./data_AFI/AFI_zeo.csv', index=False)
AFI_syn.to_csv('./data_AFI/AFI_syn.csv', index=False)

741 191
len of test_smiles: 317
len of test_zeo: 317
len of test_syn: 317
type of test_smiles: <class 'list'>
type of test_zeo: <class 'list'>
type of test_syn: <class 'list'>


In [6]:
# load the data
AFI_smiles = read_strings('./data_AFI/AFI_smiles.csv', idx=False)
AFI_zeo = read_vec('./data_AFI/AFI_zeo.csv', idx=False)
AFI_syn = read_vec('./data_AFI/AFI_syn.csv', idx=False)

vocab = WordVocab.load_vocab('./model_hub/vocab.pkl')
print('the vocab size is :', len(vocab))

the vocab size is : 45


In [7]:
# create the dataset and dataloader
AFI_dataset = Seq2seqDataset(AFI_zeo, AFI_syn, AFI_smiles, vocab)
# test Seq2seqDataset
for i, (zeo, syn, smiles) in enumerate(AFI_dataset):
    print('zeo:', zeo)
    print('syn:', syn)
    print('smiles:', smiles)
    print('zeo shape:', zeo.shape)
    print('syn shape:', syn.shape)
    print('smiles shape:', smiles.shape)
    print('type of smiles:', type(smiles))
    print('type of zeo:', type(zeo))
    print('type of syn:', type(syn))
    break

AFI_dataloader = DataLoader(AFI_dataset, batch_size=batch_size, shuffle=True)
# test Seq2seqDataset
for i, (zeo, syn, smiles) in enumerate(AFI_dataloader):
    print('zeo:', zeo)
    print('syn:', syn)
    print('smiles:', smiles)
    print('zeo shape:', zeo.shape)
    print('syn shape:', syn.shape)
    print('smiles shape:', smiles.shape)
    break

zeo: tensor([0.6940, 0.2500, 0.3333, 0.1397, 0.3090, 0.3304, 0.3304])
syn: tensor([0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
smiles: tensor([ 3,  6, 11,  6,  6,  7, 12,  7,  6,  8,  6, 13,  6,  6,  6,  6,  6, 13,
         8,  6,  6,  6, 11,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 

In [8]:
# get CHA code file
# read data
data = pd.read_excel(data_file, engine='openpyxl')
smiles_aug, zeo_features_aug, syn_features_aug, codes_aug = data_augment(augment=False, data=data)

zeo_features = np.array(zeo_features_aug)
zeo_features = norm(zeo_features)

smiles, zeo_vectors, syn_vectors, codes = data_sift(smiles_aug, zeo_features, syn_features_aug, codes_aug)
print(len(set(smiles)), len(set(codes)))
unique_codes = ['CHA']
split = "CHA"
_, _, _, _, test_smiles, test_zeo, test_syn, _ = data_split(split, unique_codes,
                                                                        smiles, zeo_vectors, syn_vectors, codes)

print('len of test_smiles:', len(test_smiles))
print('len of test_zeo:', len(test_zeo))
print('len of test_syn:', len(test_syn))

print('type of test_smiles:', type(test_smiles))
print('type of test_zeo:', type(test_zeo))
print('type of test_syn:', type(test_syn))

test_smiles = test_smiles * 250
test_zeo = test_zeo * 250  # * 121 42
test_syn = test_syn * 250  # * 121 42
CHA_smiles = test_smiles[0:1000]
CHA_zeo = test_zeo[0:1000]
CHA_syn = test_syn[0:1000]

# save CHA_smiles, CHA_zeo, CHA_syn to data_CHA by .csv
CHA_smiles = pd.DataFrame(CHA_smiles)
CHA_zeo = pd.DataFrame(CHA_zeo)
CHA_syn = pd.DataFrame(CHA_syn)
CHA_smiles.to_csv('./data_CHA/CHA_smiles.csv', index=False)
CHA_zeo.to_csv('./data_CHA/CHA_zeo.csv', index=False)
CHA_syn.to_csv('./data_CHA/CHA_syn.csv', index=False)

741 191
len of test_smiles: 243
len of test_zeo: 243
len of test_syn: 243
type of test_smiles: <class 'list'>
type of test_zeo: <class 'list'>
type of test_syn: <class 'list'>


In [9]:
# get AEI code file
# read data
data = pd.read_excel(data_file, engine='openpyxl')
smiles_aug, zeo_features_aug, syn_features_aug, codes_aug = data_augment(augment=False, data=data)

zeo_features = np.array(zeo_features_aug)
zeo_features = norm(zeo_features)

smiles, zeo_vectors, syn_vectors, codes = data_sift(smiles_aug, zeo_features, syn_features_aug, codes_aug)
print(len(set(smiles)), len(set(codes)))
unique_codes = ['AEI']
split = "AEI"
_, _, _, _, test_smiles, test_zeo, test_syn, _ = data_split(split, unique_codes,
                                                                        smiles, zeo_vectors, syn_vectors, codes)

print('len of test_smiles:', len(test_smiles))
print('len of test_zeo:', len(test_zeo))
print('len of test_syn:', len(test_syn))

print('type of test_smiles:', type(test_smiles))
print('type of test_zeo:', type(test_zeo))
print('type of test_syn:', type(test_syn))

test_smiles = test_smiles * 250
test_zeo = test_zeo * 250  # * 121 42
test_syn = test_syn * 250  # * 121 42
AEI_smiles = test_smiles[0:1000]
AEI_zeo = test_zeo[0:1000]
AEI_syn = test_syn[0:1000]

# save AEI_smiles, AEI_zeo, AEI_syn to data_AEI by .csv
AEI_smiles = pd.DataFrame(AEI_smiles)
AEI_zeo = pd.DataFrame(AEI_zeo)
AEI_syn = pd.DataFrame(AEI_syn)
AEI_smiles.to_csv('./data_AEI/AEI_smiles.csv', index=False)
AEI_zeo.to_csv('./data_AEI/AEI_zeo.csv', index=False)
AEI_syn.to_csv('./data_AEI/AEI_syn.csv', index=False)

741 191
len of test_smiles: 83
len of test_zeo: 83
len of test_syn: 83
type of test_smiles: <class 'list'>
type of test_zeo: <class 'list'>
type of test_syn: <class 'list'>
